In [1]:
# Problem Statement- Predicting Survival in the Titanic Data Set

In [2]:
# import the packages

import numpy as np
import pandas as pd
import seaborn as sb
import sklearn
from pandas import Series, DataFrame
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn import tree, metrics, model_selection, preprocessing
from sklearn.model_selection import cross_val_score

In [3]:
url= "https://raw.githubusercontent.com/BigDataGal/Python-for-Data-Science/master/titanic-train.csv"

titanic = pd.read_csv(url)

In [4]:
# Making sex as 0 AND 1
data = [titanic]
genders = {"male": 0, "female": 1}
for dataset in data:
    dataset['Sex'] = dataset['Sex'].map(genders)

In [5]:
# Insert the null value in age with value
data = [titanic]
for dataset in data:
    mean = titanic["Age"].mean()
    std = titanic["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = titanic["Age"].astype(int)

In [6]:
# Make the age in specific group
data = [titanic]
for dataset in data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 27), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age'] = 6
    dataset.loc[ dataset['Age'] > 66, 'Age'] = 6

In [7]:
# Fill the fare with 0 where null and make it integer type
data = [titanic]
for dataset in data:
    dataset['Fare'] = dataset['Fare'].fillna(0)
    dataset['Fare'] = dataset['Fare'].astype(int)

In [8]:
# Drop the columns which is not needed for prediction
titanic = titanic.drop(['PassengerId'], axis=1)
titanic = titanic.drop(['Name'], axis=1)
titanic = titanic.drop(['Ticket'], axis=1)
titanic = titanic.drop(['Cabin'], axis=1)
titanic = titanic.drop(['Embarked'], axis=1)

In [9]:
# Get X(independent variable) and Y(dependent variable)
X = titanic.drop("Survived", axis=1)
Y = titanic["Survived"]

In [10]:
# split data randomly into 70% training and 30% test
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.3, random_state=0)

In [11]:
#train the decision tree
dtree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3,random_state=0)
dtree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [12]:
# use the model to make predictions with the test data
y_pred = dtree.predict(X_test)

In [14]:
acc_decision_tree = round(dtree.score(X_train, y_train) * 100, 2)
print('Decision Tree score',acc_decision_tree)

Decision Tree score 81.38


In [17]:
print('Confusion matrix:')
predictions = cross_val_predict(dtree, X_train, y_train, cv=3)
print(confusion_matrix(y_train, predictions))

Confusion matrix:
[[349  32]
 [101 141]]


In [ ]:
print('F1 score is',f1_score(y_train, predictions))

In [20]:
y_scores = dtree.predict_proba(X_train)
y_scores = y_scores[:,1]

In [ ]:
# how did our model perform?
accuracy = metrics.accuracy_score(y_test, y_pred)
print('Accuracy: {:.2f}'.format(accuracy))

In [24]:
scores = cross_val_score(estimator= dtree, # Model to test
X= X_train,
y = y_train, # Target variable
scoring = "accuracy", # Scoring metric
cv=10) # Cross validation folds
print("Accuracy per fold: ")
print(scores)
print("Average accuracy: ", scores.mean())

Accuracy per fold: 
[0.78125    0.80952381 0.80645161 0.79032258 0.79032258 0.82258065
 0.79032258 0.83870968 0.83870968 0.87096774]
Average accuracy:  0.8139160906298002
